In [12]:
import pandas as pd
import json

#  working backwards starting with the most recent data

In [13]:
df_02_2024 = pd.read_csv("./bikes_data/trips_data/202402-bluebikes-tripdata.csv")
df_02_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231947 entries, 0 to 231946
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             231947 non-null  object 
 1   rideable_type       231947 non-null  object 
 2   started_at          231947 non-null  object 
 3   ended_at            231947 non-null  object 
 4   start_station_name  231653 non-null  object 
 5   start_station_id    231653 non-null  object 
 6   end_station_name    231252 non-null  object 
 7   end_station_id      231252 non-null  object 
 8   start_lat           231947 non-null  float64
 9   start_lng           231947 non-null  float64
 10  end_lat             231781 non-null  float64
 11  end_lng             231781 non-null  float64
 12  member_casual       231947 non-null  object 
dtypes: float64(4), object(9)
memory usage: 23.0+ MB


In [14]:
df_02_2024.head(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,8B2BB46B153DFF46,classic_bike,2024-02-26 15:59:59,2024-02-26 16:19:59,Boylston St at Massachusetts Ave,B32018,Main St at Austin St,D32020,42.347406,-71.086784,42.375603,-71.064608,casual
1,B52240FE687397A2,classic_bike,2024-02-24 08:55:34,2024-02-24 09:02:09,Lansdowne T Stop,B32058,1200 Beacon St,K32015,42.347345,-71.100168,42.344149,-71.114674,member
2,773A7C56ADD0B833,classic_bike,2024-02-27 16:29:47,2024-02-27 17:02:02,Purchase St at Pearl St,A32026,1200 Beacon St,K32015,42.354659,-71.053181,42.344149,-71.114674,member
3,B5D88733E5D33527,electric_bike,2024-02-11 21:30:47,2024-02-11 21:40:42,Main St at Austin St,D32020,Third at Binney,M32048,42.375629,-71.064694,42.365445,-71.082771,member
4,9E172008FB4A665F,classic_bike,2024-02-03 11:47:14,2024-02-03 12:13:21,Encore,V32005,Encore,V32005,42.393293,-71.072448,42.393293,-71.072448,member
5,017A80320F345591,classic_bike,2024-02-02 16:39:41,2024-02-02 17:06:33,Encore,V32005,Encore,V32005,42.393293,-71.072448,42.393293,-71.072448,member
6,0BEAF07D2181FFF1,electric_bike,2024-02-14 17:07:43,2024-02-14 17:42:46,Purchase St at Pearl St,A32026,Purchase St at Pearl St,A32026,42.354780,-71.053210,42.354659,-71.053181,casual
7,9BFE2E331B981007,electric_bike,2024-02-20 07:14:52,2024-02-20 07:23:11,Main St at Austin St,D32020,Surface Rd at Summer St,A32042,42.375672,-71.064712,42.352946,-71.056564,member
8,D25D29A38A7F60FA,electric_bike,2024-02-20 16:17:36,2024-02-20 16:28:56,Surface Rd at Summer St,A32042,Main St at Austin St,D32020,42.352834,-71.057070,42.375603,-71.064608,member
9,01A09CDEE178F5F1,electric_bike,2024-02-19 10:03:16,2024-02-19 10:12:09,Main St at Austin St,D32020,Purchase St at Pearl St,A32026,42.375664,-71.064710,42.354659,-71.053181,member


In [15]:
# splits a datetime string into the compononates of year, month, day, hour, min, second
# must be in the form "year-month-day hour:min:second"  
def date_time_dict(datetime):
    datetime = datetime.split()
    date = datetime[0]
    time = datetime[1]
    year, month, day = date.split("-") 
    hour, min, second = time.split(":")
    return {"year":year, "month":month, "day":day, "hour":hour, "minuet":min, "second":second}

In [18]:
def get_station(long, lat, station_dict, df, i):
    start_coordinates = str(long) + " " + str(lat)

    if start_coordinates not in station_dict:
        start_station_name = df.loc[i, "start_station_name"]
        start_station_id = df.loc[i, "start_station_id"]
        station_dict[start_coordinates] = {"name": start_station_name, "id": start_station_id, "lng": long, "lat": lat}

    return station_dict[start_coordinates], station_dict

In [19]:
doc_array = []
station_dict = {}

for i in range(len(df_02_2024)):
    doc = {}
    doc["ride_id"] = df_02_2024.loc[i, "ride_id"]
    doc["rideable_type"] = df_02_2024.loc[i, "rideable_type"]
    doc["started_at"] = date_time_dict(df_02_2024.loc[i, "started_at"])
    doc["ended_at"] = date_time_dict(df_02_2024.loc[i, "ended_at"])

    start_lat = df_02_2024.loc[i, "start_lat"]
    start_lng = df_02_2024.loc[i, "start_lng"]
    start_station, station_dict = get_station(long=start_lng, lat=start_lat, station_dict=station_dict, df=df_02_2024, i=i)
    end_lat = df_02_2024.loc[i, "end_lat"]
    end_lng = df_02_2024.loc[i, "end_lng"]
    end_station, station_dict = get_station(long=end_lng, lat=end_lat, station_dict=station_dict, df=df_02_2024, i=i)
    doc["start_station"] = start_station
    doc["end_station"] = end_station
    
    doc["membership"] = df_02_2024.loc[i, "member_casual"]
    
    doc_array.append(doc)

In [21]:
with open("2024_02_trips.txt", "w") as file:
    file.write(str(doc_array))

with open("station_dict.txt", "w") as file:
    file.write(str(station_dict))